# <font color='red'>Importing Necessary Libraries</font>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install chart_studio
!pip install textstat

# Visualisation Library
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#visualisation libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')


#ml
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
#standard libraries
import emoji
import re
import string

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     |████████████████████████████████| 64 kB 1.8 MB/s 
     |████████████████████████████████| 102 kB 3.8 MB/s 
     |████████████████████████████████| 3.0 MB 14.4 MB/s 


/kaggle/input/indonesiandata/emotion_id_tweets.json


In [2]:
df_new=pd.read_json('/kaggle/input/indonesiandata/emotion_id_tweets.json',lines=True)
df_new.head()

,content,annotation,extras,metadata
0,@04Nakula @jokowi @prabowo @Gerindra @Dailymot...,"{'labels': ['joy'], 'note': ''}",NaN,"{'first_done_at': 1527710555000, 'last_updated..."
1,@Achmadr06327297 @fadlizon @Gerindra @prabowo ...,"{'labels': ['joy'], 'note': ''}",NaN,"{'first_done_at': 1527984453000, 'last_updated..."
2,@Gerindra @PKSejahtera @prabowo Seandainya sek...,"{'labels': ['fear'], 'note': ''}",NaN,"{'first_done_at': 1528396641000, 'last_updated..."
3,@fadlizon @Gerindra @prabowo Saya dukung pak p...,"{'labels': ['joy'], 'note': ''}",NaN,"{'first_done_at': 1527988971000, 'last_updated..."
4,@Gerindra @PKSejahtera Mencari yang setia dan ...,"{'labels': ['trust'], 'note': ''}",NaN,"{'first_done_at': 1528445338000, 'last_updated..."


In [3]:
ex_label=[]
for i in range(df_new['annotation'].shape[0]):
    if df_new['annotation'][i]['labels']==[]:
        ex_label.append('no_emotion')
    else:
        ex_label.append(df_new.annotation[i]['labels'][0])

In [4]:
df=pd.DataFrame()
df['content']=df_new['content']
df['emotion']=ex_label
df.head()

,content,emotion
0,@04Nakula @jokowi @prabowo @Gerindra @Dailymot...,joy
1,@Achmadr06327297 @fadlizon @Gerindra @prabowo ...,joy
2,@Gerindra @PKSejahtera @prabowo Seandainya sek...,fear
3,@fadlizon @Gerindra @prabowo Saya dukung pak p...,joy
4,@Gerindra @PKSejahtera Mencari yang setia dan ...,trust


## First Look

In [5]:
df.describe()

,content,emotion
count,8682,8682
unique,6462,9
top,Grace Natalie kemudian membandingkan hasil tem...,joy
freq,18,2327


In [6]:
df['len']=df['content'].astype(str).apply(len)
df.head()

,content,emotion,len
0,@04Nakula @jokowi @prabowo @Gerindra @Dailymot...,joy,116
1,@Achmadr06327297 @fadlizon @Gerindra @prabowo ...,joy,116
2,@Gerindra @PKSejahtera @prabowo Seandainya sek...,fear,129
3,@fadlizon @Gerindra @prabowo Saya dukung pak p...,joy,89
4,@Gerindra @PKSejahtera Mencari yang setia dan ...,trust,117


Removed Duplicate Content

In [7]:
df=df.drop_duplicates(subset='content',keep='first')
df.shape

(6462, 3)

## <font color='purple'>MISSING VALUES</font>

In [8]:
def missing_value_of_data(data):
    total=data.isnull().sum().sort_values(ascending=False)
    percentage=round(total/data.shape[0]*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])
missing_value_of_data(df)


,Total,Percentage
len,0,0.0
emotion,0,0.0
content,0,0.0


No Missing Values In 3 columns

## <font color='purple'>Percentage Of Emotions :) </font>

In [9]:
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])
count_values_in_column(df,'emotion')


,Total,Percentage
joy,1776,27.48
trust,1447,22.39
anticipation,950,14.70
anger,833,12.89
disgust,491,7.60
sadness,445,6.89
fear,430,6.65
surprise,75,1.16
no_emotion,15,0.23


## <font color='Green'>Percentage Of Emotions</font>

In [10]:
name_of_emotion=df.emotion.unique()
value_of_emotion=list(df.emotion.value_counts())

fig=go.Figure([go.Bar(x=list(name_of_emotion), y=value_of_emotion)])
fig.show()

# <font color='purple'>N Gram Charts</font> 

In [11]:
!pip install translators

  Created wheel for translators: filename=translators-4.4.3-py3-none-any.whl size=12577 sha256=b7afcd7cfebdd0f113fc79fd318be61b5425cafdb0d63fb663eec0da51dbcb8c
  Stored in directory: /root/.cache/pip/wheels/4e/ff/46/302947550b9be3b3e33f59197d89e6a55c6bd94ddeee1b7b10
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14588 sha256=e7b02b2e8b175a6421d3f40e7b70dd9c3a3f1ac1a31e8ae885adb8b7e6a6f8f2
  Stored in directory: /root/.cache/pip/wheels/9a/ee/03/da5c0b4a8c13362beeb844eb913bbe58a89bde1de2b9157007
Successfully built translators PyExecJS


In [12]:
import translators as ts

In [13]:
def ngrams_top(corpus,ngram_range,n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english',ngram_range=ngram_range).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    total_list=words_freq[:n]
    df=pd.DataFrame(total_list,columns=['text','count'])
    
    english_ngram=[] #translating indonesian to english
    for i  in range(10):
        english_ngram.append(ts.google(df['text'][i], 'auto', 'en'))
      
 
    # Plotting Grams and their english conversion   
    fig = make_subplots(rows=1, cols=2)
    fig.add_trace(
        go.Bar(x=df['count'][::-1],
                    y=df['text'][::-1],
                   name='Indonesian',
                    marker_color='rgb(55, 83, 109)',
                    orientation='h'
                    ),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(x=df['count'][::-1],
                    y=english_ngram[::-1],
                   name='English',
                    marker_color='rgb(123, 67, 199)',
                    orientation='h'
                    ),
        row=1, col=2
    )
    fig.update_layout(height=600, width=2000, title_text=str(ngram_range[0])+" grams for Indonesian/English")
    fig.show()

### <font color='red'>1 Gram</font>

In [14]:
ngrams_top(df['content'],(1,1),n=10)


UseTimeSeconds(fn: google_api): 0.23
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.2
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.17


## <font color='red'>2 Gram</font>

In [15]:
ngrams_top(df['content'],(2,2),n=10)


UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.18


## <font color='red'>3 Gram</font>

In [16]:
ngrams_top(df['content'],(3,3),n=10)

UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.15


# <font color='brown'>Sentence Length Analysis</font>

In [17]:
joy=df[df['emotion']=='joy']
trust=df[df['emotion']=='trust']
fear=df[df['emotion']=='fear']
anger=df[df['emotion']=='anger']
anticipation=df[df['emotion']=='anticipation']
disgust=df[df['emotion']=='disgust']
sadness=df[df['emotion']=='sadness']
surprise=df[df['emotion']=='surprise']

In [18]:
joy['len'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='red',
    yTitle='count',
    title='Joy Text Length Distribution')
trust['len'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='green',
    yTitle='count',
    title='Trust Text Length Distribution')

fear['len'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='pink',
    yTitle='count',
    title='Fear Text Length Distribution')

anger['len'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='orange',
    yTitle='count',
    title='Anger Text Length Distribution')

anticipation['len'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    yTitle='count',
    title='Anticipation Text Length Distribution')
disgust['len'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    
    yTitle='count',
    title='Disgust Text Length Distribution')
sadness['len'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='gold',
    yTitle='count',
    title='Sadness Text Length Distribution')
surprise['len'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='blue',
    yTitle='count',
    title='Surprise Text Length Distribution')

**Histogram clearly shows:-
**
* Joy , Trust, Fear, anger,Anticipation, Disgust emotions have comment length Mostly less than 600 words.
* Sadness has Comment Length msotly less than 800 words.
* Comments having Surprise emotion is Mostly SKewed.

# <font color='Orange'>Most used Pair of words in each emotion</font>

In [19]:
def ngrams_tops(corpus,ngram_range,emotions,n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english',ngram_range=ngram_range).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    total_list=words_freq[:n]
    df=pd.DataFrame(total_list,columns=['text','count'])
    
    english_ngram=[] #translating indonesian to english
    for i  in range(10):
        english_ngram.append(ts.google(df['text'][i], 'auto', 'en'))
      
 
    # Plotting Grams and their english conversion   
    fig = make_subplots(rows=1, cols=2)
    fig.add_trace(
        go.Bar(x=df['count'][::-1],
                    y=df['text'][::-1],
                   name='Indonesian',
                    marker_color='rgb(55, 83, 109)',
                    orientation='h'
                    ),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(x=df['count'][::-1],
                    y=english_ngram[::-1],
                   name='English',
                    marker_color='rgb(123, 67, 199)',
                    orientation='h'
                    ),
        row=1, col=2
    )
    fig.update_layout(height=600, width=2000,title_text="Most Used words in "+str(emotions).capitalize())
    fig.show()

    
def most_used(emotions):
    emotion=df.loc[df['emotion']==str(emotions)]

    _1gram_emotion=ngrams_tops(emotion['content'],(1,1),emotions,n=10)
    
    return _1gram_emotion
   
        

## 1.JOY

In [20]:
most_used('joy')

UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.2
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.16


## 2. Fear

In [21]:
most_used('fear')

UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.23
UseTimeSeconds(fn: google_api): 0.16


## 3. Trust

In [22]:
most_used('trust')

UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.15


## <font color='green'>4. Surprise</font>

In [23]:
most_used('surprise')

UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.14
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.16


## <font color='orange'>5. Anger </font>

In [24]:
most_used('anger')

UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.2
UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.14
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.15


## 6.Anticipation

In [25]:
most_used('anticipation')

UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.14
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.18


## <font color='purple'>7. Disgust </font>

In [26]:
most_used('disgust')

UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.26
UseTimeSeconds(fn: google_api): 0.15


## <font color='red'>8. Sadness </font>

In [27]:
most_used('sadness')

UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.16
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.18
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.17
UseTimeSeconds(fn: google_api): 0.19
UseTimeSeconds(fn: google_api): 0.15
UseTimeSeconds(fn: google_api): 0.17


## Observation from Above 

* when we observed most used **single** words in each emotion they were mostly :-
        stopwords like=we,for,me,first,can  etc
* Observation from Most Used Bigrams in each motion got us some keyowrds:-
      1. People Were Talking about their personal belongings while expressing Sadness and Fear
      example= Facebook Account,Account etc.
      
      2. Surprise emotion mostly had the words like:-
      CFC boxing and Chicken pieces after further analysis we can draw some conclusions from data
      
      
        
* Fun Fact :-
        Disgust and Anger was always conveyed in Native Indonesian( that even Google Translate couldn't translate)

# <font color='Blue'>Preprocessing Function</font>

## <font color='purple'>Emoticons <font>

Convert Emoji To Text

In [28]:
def find_emoji(text):
    emo_text=emoji.demojize(text)
    line=re.findall(r'\:(.*?)\:',emo_text)
    return line
sentence="I love ⚽ very much 😁"
find_emoji(sentence)

# Emoji cheat sheet - https://www.webfx.com/tools/emoji-cheat-sheet/
# Uniceode for all emoji : https://unicode.org/emoji/charts/full-emoji-list.html

['soccer_ball', 'beaming_face_with_smiling_eyes']

This Helper function could be used somewhere else but will not use it here

Because our text is in Indonesian and after converting emoji to text we will get **english text** and Hence will spoil our Training data

#### Removing Emoji from Text

In [29]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

sentence="Its all about \U0001F600 face"
print(sentence)
remove_emoji(sentence)

Its all about 😀 face


'Its all about  face'

In [30]:
df['content']=df['content'].apply(lambda x: remove_emoji(x))

## <font color='purple'>Repetitive Characters </font>

Removing Repetitve Characters

Note :we will use this function in our code

In [31]:
def rep(text):
    grp = text.group(0)
    if len(grp) > 1:
        return grp[0:1] # can change the value here on repetition
    return grp
   
def unique_char(rep,sentence):
    convert = re.sub(r'(\w)\1+', rep, sentence) 
    return convert

sentence="heyyy this is loooong textttt"
unique_char(rep,sentence)


'hey this is long text'

In [32]:
df['content']=df['content'].apply(lambda x : unique_char(rep,x))